# RNN

In [1]:
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
text = """곱다
경마장에 있는 말이 뛰고 있다\n
그의 말이 법이다.\n
가는 말이 고와야 오는 말이 \n"""

In [3]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
vocab_size = len(tokenizer.word_index) + 1
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 12


In [4]:
print(tokenizer.word_index)

{'말이': 1, '곱다': 2, '경마장에': 3, '있는': 4, '뛰고': 5, '있다': 6, '그의': 7, '법이다': 8, '가는': 9, '고와야': 10, '오는': 11}


In [5]:
sequences = list()
for line in text.split('\n'): # 줄 바꿈 문자를 기준으로 문장 토큰화
    encoded = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[: i+1]
        sequences.append(sequence)

print('학습에 사용할 샘플의 개수: %d' % len(sequences))

학습에 사용할 샘플의 개수: 10


In [6]:
print(sequences)

[[3, 4], [3, 4, 1], [3, 4, 1, 5], [3, 4, 1, 5, 6], [7, 1], [7, 1, 8], [9, 1], [9, 1, 10], [9, 1, 10, 11], [9, 1, 10, 11, 1]]


In [7]:
max_len = max(len(l) for l in sequences) # 모든 샘플에서 길이가 가장 긴 샘플의 길이 출력
print('샘플의 최대 길이 : {}'.format(max_len))

샘플의 최대 길이 : 5


In [8]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')

In [9]:
print(sequences)

[[ 0  0  0  3  4]
 [ 0  0  3  4  1]
 [ 0  3  4  1  5]
 [ 3  4  1  5  6]
 [ 0  0  0  7  1]
 [ 0  0  7  1  8]
 [ 0  0  0  9  1]
 [ 0  0  9  1 10]
 [ 0  9  1 10 11]
 [ 9  1 10 11  1]]


In [10]:
sequences = np.array(sequences)
X = sequences[:, :-1]
y = sequences[:, -1]

In [11]:
print(X)

[[ 0  0  0  3]
 [ 0  0  3  4]
 [ 0  3  4  1]
 [ 3  4  1  5]
 [ 0  0  0  7]
 [ 0  0  7  1]
 [ 0  0  0  9]
 [ 0  0  9  1]
 [ 0  9  1 10]
 [ 9  1 10 11]]


In [12]:
print(y)

[ 4  1  5  6  1  8  1 10 11  1]


In [13]:
y = to_categorical(y, num_classes=vocab_size)

In [14]:
print(y)

[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


## 2) 모델 설계하기
RNN 모델에 데이터를 훈련시키기

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

In [16]:
embedding_dim = 10
hidden_units = 32

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(SimpleRNN(hidden_units))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=2)

Epoch 1/200
1/1 - 2s - loss: 2.4776 - accuracy: 0.0000e+00 - 2s/epoch - 2s/step
Epoch 2/200
1/1 - 0s - loss: 2.4633 - accuracy: 0.1000 - 16ms/epoch - 16ms/step
Epoch 3/200
1/1 - 0s - loss: 2.4488 - accuracy: 0.2000 - 16ms/epoch - 16ms/step
Epoch 4/200
1/1 - 0s - loss: 2.4343 - accuracy: 0.3000 - 15ms/epoch - 15ms/step
Epoch 5/200
1/1 - 0s - loss: 2.4195 - accuracy: 0.4000 - 14ms/epoch - 14ms/step
Epoch 6/200
1/1 - 0s - loss: 2.4043 - accuracy: 0.5000 - 16ms/epoch - 16ms/step
Epoch 7/200
1/1 - 0s - loss: 2.3888 - accuracy: 0.5000 - 16ms/epoch - 16ms/step
Epoch 8/200
1/1 - 0s - loss: 2.3728 - accuracy: 0.5000 - 17ms/epoch - 17ms/step
Epoch 9/200
1/1 - 0s - loss: 2.3562 - accuracy: 0.5000 - 16ms/epoch - 16ms/step
Epoch 10/200
1/1 - 0s - loss: 2.3389 - accuracy: 0.5000 - 15ms/epoch - 15ms/step
Epoch 11/200
1/1 - 0s - loss: 2.3209 - accuracy: 0.5000 - 15ms/epoch - 15ms/step
Epoch 12/200
1/1 - 0s - loss: 2.3020 - accuracy: 0.5000 - 18ms/epoch - 18ms/step
Epoch 13/200
1/1 - 0s - loss: 2.2823 

In [17]:
def sentence_generation(model, tokenizer, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word
    sentence = ''

    # n번 반복
    for _ in range(n):
        # 현재 단어에 대한 정수 인코딩과 패딩
        encoded = tokenizer.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=5, padding='pre')
        # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장.
        result = model.predict(encoded, verbose=0)
        result = np.argmax(result, axis=1)

        for word, index in tokenizer.word_index.items(): 
            # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면 break
            if index == result:
                break

        # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        current_word = current_word + ' '  + word

        # 예측 단어를 문장에 저장
        sentence = sentence + ' ' + word

    sentence = init_word + sentence
    return sentence

In [18]:
print(sentence_generation(model, tokenizer, '경마장에', 4))

경마장에 말이 말이 뛰고 있다


In [19]:
print(sentence_generation(model, tokenizer, '가는', 5))

가는 말이 고와야 오는 말이 고와야


# LSTM을 이용하여 텍스트 생성하기

In [20]:
import numpy as np
import pandas as pd
import pandas as pad_sequences
from string import punctuation

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [21]:
df = pd.read_excel(r'C:/Users/GJAISCHOOL/Desktop/X_filter/Algorithm/dataset/Sentence data.xlsx')
df.head()

,Sentence,label
0,좌배 까는건 ㅇㅂ,1
1,집에 롱 패딩만 세 개다. 10년 더 입어야지 ㅋㅋ,0
2,개소리야 니가 빨갱이를 옹호하고 드루킹을 ㅇㅇ짓이라고 말못해서 삐진거야 빨갱아,1
3,세탁이라고 봐도 된다,0
4,애새끼가 초딩도 아니고 ㅋㅋㅋㅋ,1


In [22]:
print('열의 개수 : ', len(df.columns))
print(df.columns)

열의 개수 :  2
Index(['Sentence', 'label'], dtype='object')


In [23]:
print(df['Sentence'].isnull().values.any())

False


In [25]:
headline = []
# 헤드라인의 값들을 리스트로 저장
headline.extend(list(df.Sentence.values))
headline[:5]

['좌배 까는건 ㅇㅂ',
 '집에 롱 패딩만 세 개다. 10년 더 입어야지 ㅋㅋ',
 '개소리야 니가 빨갱이를 옹호하고 드루킹을 ㅇㅇ짓이라고 말못해서 삐진거야 빨갱아',
 '세탁이라고 봐도 된다',
 '애새끼가 초딩도 아니고 ㅋㅋㅋㅋ']

In [26]:
print('총 샘플의 개수 : {}'.format(len(headline)))

총 샘플의 개수 : 28737


In [27]:
headline = [word for word in headline if word != "Unkown"]
print('노이즈값 제거 후 샘플의 개수 : {}'.format(len(headline)))

노이즈값 제거 후 샘플의 개수 : 28737


In [28]:
headline[:5]

['좌배 까는건 ㅇㅂ',
 '집에 롱 패딩만 세 개다. 10년 더 입어야지 ㅋㅋ',
 '개소리야 니가 빨갱이를 옹호하고 드루킹을 ㅇㅇ짓이라고 말못해서 삐진거야 빨갱아',
 '세탁이라고 봐도 된다',
 '애새끼가 초딩도 아니고 ㅋㅋㅋㅋ']

In [29]:
def repreprocessing(raw_sentence):
    preproceseed_sentence = raw_sentence.encode("utf8").decode("ascii",'ignore')
    # 구두점 제거와 동시에 소문자화
    return ''.join(word for word in preproceseed_sentence if word not in punctuation).lower()

preprocessed_headline = [repreprocessing(x) for x in headline]
preprocessed_headline[:5]

['  ', '     10   ', '        ', '  ', '   ']

In [30]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(preprocessed_headline)
vocab_size = len(tokenizer.word_index) + 1
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 1059


In [31]:
sequences = list()

for sentence in preprocessed_headline:
    # 각 샘플에 대한 정수 인코딩
    encoded = tokenizer.texts_to_sequences([sentence])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

sequences[:11]

[[6, 16],
 [10, 10],
 [10, 10, 10],
 [235, 236],
 [235, 236, 163],
 [1, 5],
 [79, 89],
 [79, 89, 32],
 [363, 13],
 [237, 364],
 [164, 15]]

In [32]:
index_to_word = {}
for key, value in tokenizer.word_index.items(): # 인덱스를 단어로 바꾸기 위해 index_to_word를 생성
    index_to_word[value] = key

print('빈도수 상위 582번 단어 : {}'.format(index_to_word[582]))

빈도수 상위 582번 단어 : httpswwwilbecomview11226188404c11226214943comm11226214943


In [33]:
max_len = max(len(l) for l in sequences)
print('샘플의 최대 길이 : {}'.format(max_len))

샘플의 최대 길이 : 53


In [53]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(sequences[:3])

[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  6 16]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0 10 10]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0 10 10 10]]


In [35]:
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]

In [36]:
print(X[: 3])

[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  6]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0 10]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0 10 10]]


In [37]:
print(y[:3])

[16 10 10]


In [38]:
y = to_categorical(y, num_classes=vocab_size)

In [39]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM


In [54]:
embedding_dim = 10
hidden_units = 128

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(LSTM(hidden_units))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=2000, verbose=2)


Epoch 1/2000
48/48 - 3s - loss: 6.2193 - accuracy: 0.0562 - 3s/epoch - 56ms/step
Epoch 2/2000
48/48 - 1s - loss: 5.3716 - accuracy: 0.0673 - 658ms/epoch - 14ms/step
Epoch 3/2000
48/48 - 1s - loss: 5.2645 - accuracy: 0.0719 - 547ms/epoch - 11ms/step
Epoch 4/2000
48/48 - 1s - loss: 5.2148 - accuracy: 0.0667 - 856ms/epoch - 18ms/step
Epoch 5/2000
48/48 - 1s - loss: 5.1676 - accuracy: 0.0680 - 837ms/epoch - 17ms/step
Epoch 6/2000
48/48 - 1s - loss: 5.1191 - accuracy: 0.0667 - 753ms/epoch - 16ms/step
Epoch 7/2000
48/48 - 1s - loss: 5.0755 - accuracy: 0.0660 - 724ms/epoch - 15ms/step
Epoch 8/2000
48/48 - 1s - loss: 5.0344 - accuracy: 0.0725 - 734ms/epoch - 15ms/step
Epoch 9/2000
48/48 - 1s - loss: 4.9968 - accuracy: 0.0719 - 738ms/epoch - 15ms/step
Epoch 10/2000
48/48 - 1s - loss: 4.9432 - accuracy: 0.0752 - 722ms/epoch - 15ms/step
Epoch 11/2000
48/48 - 1s - loss: 4.8954 - accuracy: 0.0725 - 763ms/epoch - 16ms/step
Epoch 12/2000
48/48 - 1s - loss: 4.8436 - accuracy: 0.0699 - 757ms/epoch - 16

In [55]:
def sentence_generation(model, tokenizer, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word
    sentence = ''

    # n번 반복
    for _ in range(n):
        encoded = tokenizer.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=max_len-1, padding='pre')

        # 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.
        result = model.predict(encoded, verbose=0)
        result = np.argmax(result, axis=1)

        for word, index in tokenizer.word_index.items(): 
            # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
            if index == result:
                break

        # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        current_word = current_word + ' '  + word

        # 예측 단어를 문장에 저장
        sentence = sentence + ' ' + word

    sentence = init_word + sentence
    return sentence


In [61]:
print(sentence_generation(model, tokenizer, '병신엿같네', 10))


병신엿같네 nomuhyun i no lb hyu e n the former gim


In [57]:
print(sentence_generation(model, tokenizer, 'how', 10))

how nomuhyun i no lb hyu e n the former gim
